# Grant's attempt at making sense of SegmentFlow outputs and grain statistics (a two-for-one special)

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from stl import mesh
# %load_ext autoreload
%matplotlib qt
# %matplotlib inline

In [46]:
# file = '../../segmentflow_02/stls/isolated-classes_00330.stl'
file = '../../segmentflow_02/stls/isolated-classes_00355.stl'
m = mesh.Mesh.from_file(file)
vertices = m.vectors.reshape((-1, 3))

# Remove duplicate rows
vertices = np.unique(vertices, axis=0)

center = vertices.mean(axis=0)
centered = vertices - center


# Plot setting
make_plt = True

# Testing:
# generate points on an ellipsoid
N = 50
theta = np.linspace(0, 1/2*np.pi, N)
phi = np.linspace(0, 1/2*np.pi, N)
x = 2.0*np.outer(np.cos(theta), np.sin(phi))
y = 1.0*np.outer(np.sin(theta), np.sin(phi))
z = np.outer(np.ones(np.size(theta)), np.cos(phi))
# flatten the arrays
x = x.flatten()
y = y.flatten()
z = z.flatten()
# stack the arrays
centered = np.vstack((x, y, z)).T

n = len(centered)
qs = np.zeros(n)

if make_plt:
    # Generate points on the unit sphere
    u = np.linspace(0, 2 * np.pi, 100)
    v = np.linspace(0, np.pi, 50)
    x = np.outer(np.cos(u), np.sin(v))
    y = np.outer(np.sin(u), np.sin(v))
    z = np.outer(np.ones_like(u), np.cos(v))
    points = np.stack([x.reshape(-1), y.reshape(-1), z.reshape(-1)], axis=1)

ellipsoid_scaling = 1.0

# Construct the bilinear form
# C = np.cov(centered.T)
C = centered.T @ centered / n
D, V = np.linalg.eig(C)
print(D)
D = D * ellipsoid_scaling
A = np.diag(D**(-0.5)) @ V.T
print(A)
B = A.T @ A - np.eye(3)

if make_plt:
    # Scale the points
    scaled_points = np.dot(points / np.sqrt(D), V)
    # Convert points back into original shape
    x = scaled_points[:,0].reshape(x.shape)
    y = scaled_points[:,1].reshape(y.shape)
    z = scaled_points[:,2].reshape(z.shape)

# scaled_points = (B @ point_prime)
# print(scaled_points)

[1.34212001 0.28512312 0.12275687]
[[ 0.71350587  0.27607301  0.39972855]
 [ 0.97601419 -0.23297461 -1.58125772]
 [ 0.60633058 -2.68082654  0.76923057]]


In [ ]:

# Check if / how far away from the ellipse
points = centered[:,:].T
# points = scaled_points[:,:].T

# Determine if the points are inside or outside the ellipsoid
for i,point in enumerate(points.T):
    point = point.reshape((-1, 1))
    Q = point.T @ B @ point
    qs[i] = (Q.item(0))

n_inside = np.sum(qs <= 1.0)

if make_plt:
    plt.figure()
    ax = plt.axes(projection='3d')
    ax.scatter3D(centered[:,0], centered[:,1], centered[:,2], cmap='viridis')
    ax.plot_surface(x, y, z,  rstride=4, cstride=4, alpha=0.6)
    plt.title(f'Scaling factor: {ellipsoid_scaling}, Ratio inside: {n_inside/n:.3f}')

In [73]:
print(np.max(z))
print(np.min(z))

1.3416407864998736
-1.3416407864998736


In [94]:
# Scale and rotate the points by the eigenvectors and eigenvalues

x = scaled_points[:, 0]
y = scaled_points[:, 1]
z = scaled_points[:, 2] 

fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(projection='3d')
ax.scatter(x, y, z, color='r', alpha=0.1)
ax.scatter(centered[:, 0], centered[:, 1], centered[:, 2], color='b', alpha=0.1)
# set equal axes scales
max_range = np.array([x.max()-x.min(), y.max()-y.min(), z.max()-z.min()]).max() / 2.0
mid_x = (x.max()+x.min()) * 0.5
mid_y = (y.max()+y.min()) * 0.5
mid_z = (z.max()+z.min()) * 0.5
max_range *= 1.2
ax.set_xlim(mid_x - max_range, mid_x + max_range)
ax.set_ylim(mid_y - max_range, mid_y + max_range)
ax.set_zlim(mid_z - max_range, mid_z + max_range)

# ToDo: plot surface and wireframe

# plot surface and wireframe
# ax.plot_surface(x, y, z,  rstride=4, cstride=4, color='b', alpha=0.1)
# ax.plot_wireframe(x, y, z,  rstride=4, cstride=4, color='k', alpha=0.3)
# add shadows
# ax.contour(x, y, z, zdir='x', offset=-1, cmap='Blues')
# ax.contour(x, y, z, zdir='y', offset=1, cmap='Blues')
# ax.contour(x, y, z, zdir='z', offset=-1, cmap='Blues')

0.9999999999999999


## Write finding the best ellipsoid as an optimization problem and use `cvxpy` to solve it


$$\begin{array}{cl} \text{maximize} & \log(\det(A))\\ \text{subject to} & \|A x_i + b\| \le 1, \quad i=1,...,n. \end{array}$$

### This next section is meant to be a good standalone

In [36]:
import cvxpy as cvx
import numpy as np
import matplotlib.pyplot as plt
from stl import mesh
# %load_ext autoreload
# %matplotlib qt
%matplotlib inline

In [31]:
# file = '../../segmentflow_02/stls/isolated-classes_00330.stl'
file = '../../segmentflow_02/stls/isolated-classes_00355.stl'
m = mesh.Mesh.from_file(file)
vertices = m.vectors.reshape((-1, 3))

# Remove duplicate rows
vertices = np.unique(vertices, axis=0)

n = vertices.shape[0]

A = cvx.Variable((3, 3), PSD=True)
b = cvx.Variable((3,1))
obj = cvx.Maximize(cvx.log_det(A))
constraints = [ cvx.norm(A@vertices[i:i+1,:].T + b) <= 1 for i in range(n) ]

In [38]:
prob = cvx.Problem(obj, constraints)
prob.solve(solver='SCS', verbose=False)

print("status:", prob.status)

# Compute constraint violation using final value of A and b:
violation = [np.linalg.norm(A.value@vertices[i:i+1,:].T + b.value) - 1 for i in range(n)]
print("constraint violation max: ", np.max(violation))


# Ellipsoid Size:
D,V = np.linalg.eig(A.value)

print("Principal axes:")
print(D)

status: optimal
constraint violation max:  0.0006510649454853912
Principal axes:
[10.13152363 17.94894171 23.20949712]


In [37]:
N = 50
theta = np.linspace(0, 2*np.pi, N)
phi = np.linspace(0, np.pi, N)
x = np.outer(np.cos(theta), np.sin(phi))
y = np.outer(np.sin(theta), np.sin(phi))
z = np.outer(np.ones(np.size(theta)), np.cos(phi))

# flatten the arrays
x = x.flatten()
y = y.flatten()
z = z.flatten()

# stack the arrays
points = np.vstack((x-b.value[0,0] , y-b.value[1,0], z-b.value[2,0]))
ellipse_points = np.linalg.solve(A.value, points)

# Reshape meshgrid style
ellipse_points = ellipse_points.reshape((3, N, N))

plt.figure()
ax = plt.axes(projection='3d')
ax.scatter3D(vertices[:,0], vertices[:,1], vertices[:,2], cmap='viridis')
ax.plot_surface(ellipse_points[0,:,:], ellipse_points[1,:,:], ellipse_points[2,:,:],  rstride=4, cstride=4, alpha=0.4, cmap='viridis')
# Make the same plots but project onto the 3 Planes, using a subplot for each plane
fig, ax = plt.subplots(1, 3, figsize=(12, 7))
ax[0].scatter(vertices[:,0], vertices[:,1], cmap='viridis')
ax[0].plot(ellipse_points[0,:,:], ellipse_points[1,:,:], alpha=0.2)
ax[0].set_xlabel('x')
ax[0].set_ylabel('y')
ax[1].scatter(vertices[:,0], vertices[:,2], cmap='viridis')
ax[1].plot(ellipse_points[0,:,:], ellipse_points[2,:,:], alpha=0.2)
ax[1].set_xlabel('x')
ax[1].set_ylabel('z')
ax[2].scatter(vertices[:,1], vertices[:,2], cmap='viridis')
ax[2].plot(ellipse_points[1,:,:], ellipse_points[2,:,:], alpha=0.2)
ax[2].set_xlabel('y')
ax[2].set_ylabel('z')
fig.suptitle('Ellipsoid Fit to Enclose 3D Points', fontsize=16)

Text(0.5, 0.98, 'Ellipsoid Fit to Enclose 3D Points')